In [3]:
import numpy as np
import random
import torch
import os
import spacy
from torchtext.vocab import GloVe
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader , SubsetRandomSampler
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

In [3]:
!gpustat

WellsFargo                Wed Sep 13 18:08:24 2023  525.125.06
[0] NVIDIA A100-SXM4-40GB | 35'C, 100 % |  3891 / 40960 MB | ajaymamidi(3254M) gdm(4M) user(4M)
[1] NVIDIA A100-SXM4-40GB | 27'C,   0 % |  1213 / 40960 MB | gdm(4M) user(4M)
[2] NVIDIA A100-SXM4-40GB | 27'C,   0 % | 31625 / 40960 MB | tonygracious(11360M) mohitsoni(19628M) gdm(4M) user(4M)
[3] NVIDIA A100-SXM4-40GB | 44'C, 100 % | 38961 / 40960 MB | tonygracious(37748M) gdm(4M) user(4M)
[4] NVIDIA A100-SXM4-40GB | 43'C, 100 % |  4163 / 40960 MB | ajaymamidi(3446M) gdm(4M) user(4M)
[5] NVIDIA A100-SXM4-40GB | 27'C,   0 % |  1911 / 40960 MB | gdm(4M) user(4M)
[6] NVIDIA A100-SXM4-40GB | 35'C, 100 % |  7911 / 40960 MB | ajaymamidi(3994M) tonygracious(3280M) gdm(4M) user(4M)
[7] NVIDIA A100-SXM4-40GB | 33'C,   0 % |  1501 / 40960 MB | gdm(4M) user(4M)


In [4]:
#Setting seed for entire notebook

def setSeed(seed = 75):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ["PYTHONHASHSEED"]=str(seed)

setSeed(75)

In [5]:
#Data Preprocessing
import io

def getCorpus(train , test , valid):
  Eng = []
  Hindi= []

  with io.open(train, encoding ='utf-8') as f:
    for line in f:
      if '\t' not in line:
        continue
      tokens = line.rstrip().split("\t")
      Eng.append(tokens[1])
      Hindi.append(tokens[0])
  with io.open(test, encoding ='utf-8') as f:
    for line in f:
      if '\t' not in line:
        continue
      tokens = line.rstrip().split("\t")
      Eng.append(tokens[1])
      Hindi.append(tokens[0])
  with io.open(valid, encoding ='utf-8') as f:
    for line in f:
      if '\t' not in line:
        continue
      tokens = line.rstrip().split("\t")
      Eng.append(tokens[1])
      Hindi.append(tokens[0])
  return Eng, Hindi
train ='data/hin_train.tsv'
test = 'data/hin_test.tsv'
valid = 'data/hin_dev.tsv'

Source , Target = getCorpus(train,test,valid)
Source = Source[:51200]
Target = Target[:51200]


In [6]:
maxSrcLen = max([len(txt) for txt in Source])
maxTgtLen = max([len(txt) for txt in Target])


In [7]:
print(f'Max Source Length : {maxSrcLen}    Max Target Length : {maxTgtLen}')
print(Source[100:110])
print(Target[100:110])

Max Source Length : 20    Max Target Length : 19
['antarrashtriye', 'antaratma', 'antratma', 'antral', 'antarashtriya', 'antarrashtriya', 'antrashtriya', 'antariksh', 'antriksh', 'antrikshyatri']
['अंतरर्राष्ट्रीय', 'अंतरात्मा', 'अंतरात्मा', 'अंतराल', 'अंतराष्ट्रीय', 'अंतराष्ट्रीय', 'अंतराष्ट्रीय', 'अंतरिक्ष', 'अंतरिक्ष', 'अंतरिक्षयात्री']


In [8]:
#Finding Source Vocabulary , making index to every word in vocab
Src_vocabulary = set()
src_word_to_idx = {}
idx=1
for word in Source:
    for i in word:
        Src_vocabulary.add(i)
        if i not in src_word_to_idx:
            src_word_to_idx[i] = idx
            idx=idx+1
Src_vocabulary.add('unk')
src_word_to_idx['unk'] = idx

idx_to_src_word = {value: key for key, value in src_word_to_idx.items()}

In [9]:
print(len(Src_vocabulary))
print(src_word_to_idx)
print(idx_to_src_word)

27
{'a': 1, 'n': 2, 'k': 3, 'g': 4, 'i': 5, 't': 6, 'u': 7, 'c': 8, 'l': 9, 'e': 10, 'r': 11, 's': 12, 'h': 13, 'd': 14, 'b': 15, 'y': 16, 'o': 17, 'j': 18, 'z': 19, 'm': 20, 'v': 21, 'w': 22, 'p': 23, 'f': 24, 'x': 25, 'q': 26, 'unk': 27}
{1: 'a', 2: 'n', 3: 'k', 4: 'g', 5: 'i', 6: 't', 7: 'u', 8: 'c', 9: 'l', 10: 'e', 11: 'r', 12: 's', 13: 'h', 14: 'd', 15: 'b', 16: 'y', 17: 'o', 18: 'j', 19: 'z', 20: 'm', 21: 'v', 22: 'w', 23: 'p', 24: 'f', 25: 'x', 26: 'q', 27: 'unk'}


In [10]:
#Finding Source Vocabulary , making index to every word in vocab
Tgt_vocabulary = set()
tgt_word_to_idx = {}
idx=1
for word in Target:
    for i in word:
        Tgt_vocabulary.add(i)
        if i not in tgt_word_to_idx:
            tgt_word_to_idx[i] = idx
            idx=idx+1
Tgt_vocabulary.add('unk')
tgt_word_to_idx['unk'] = idx

idx_to_tgt_word = {value: key for key, value in tgt_word_to_idx.items()}

In [11]:
print(len(Tgt_vocabulary))
print(tgt_word_to_idx)
print(idx_to_tgt_word)

64
{'अ': 1, 'ं': 2, 'क': 3, 'ग': 4, 'ण': 5, 'ि': 6, 'त': 7, 'ल': 8, 'ु': 9, 'र': 10, 'श': 11, 'द': 12, 'न': 13, 'े': 14, 'भ': 15, '्': 16, 'ष': 17, 'ा': 18, 'ी': 19, 'ठ': 20, 'य': 21, 'ो': 22, 'ू': 23, 'ज': 24, 'च': 25, 'म': 26, 'ट': 27, 'ड': 28, 'व': 29, 'ः': 30, 'ह': 31, 'प': 32, 'ृ': 33, 'स': 34, 'ध': 35, 'ै': 36, '़': 37, 'ब': 38, 'उ': 39, 'ॉ': 40, 'ई': 41, 'ख': 42, 'घ': 43, 'छ': 44, 'ञ': 45, 'फ': 46, 'ओ': 47, 'थ': 48, 'ढ': 49, 'झ': 50, 'ौ': 51, 'आ': 52, 'इ': 53, 'ँ': 54, 'ए': 55, 'ऊ': 56, 'ॅ': 57, 'ऋ': 58, 'ऑ': 59, 'ऐ': 60, 'औ': 61, 'ङ': 62, 'ॐ': 63, 'unk': 64}
{1: 'अ', 2: 'ं', 3: 'क', 4: 'ग', 5: 'ण', 6: 'ि', 7: 'त', 8: 'ल', 9: 'ु', 10: 'र', 11: 'श', 12: 'द', 13: 'न', 14: 'े', 15: 'भ', 16: '्', 17: 'ष', 18: 'ा', 19: 'ी', 20: 'ठ', 21: 'य', 22: 'ो', 23: 'ू', 24: 'ज', 25: 'च', 26: 'म', 27: 'ट', 28: 'ड', 29: 'व', 30: 'ः', 31: 'ह', 32: 'प', 33: 'ृ', 34: 'स', 35: 'ध', 36: 'ै', 37: '़', 38: 'ब', 39: 'उ', 40: 'ॉ', 41: 'ई', 42: 'ख', 43: 'घ', 44: 'छ', 45: 'ञ', 46: 'फ', 47: 'ओ', 48: 'थ', 49:

In [12]:
#Modifying src with index values
dataset_corpus = []
for i in range(len(Source)):
    src_ids =[src_word_to_idx[let] for let in Source[i]]
    tgt_ids =[tgt_word_to_idx[let] for let in Target[i]]
    while(len(tgt_ids)<19):
        tgt_ids.append(0)
    dataset_corpus.append((src_ids , tgt_ids))

In [13]:
print(dataset_corpus[10])

([1, 4, 1, 2, 14, 13], [1, 2, 4, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [14]:
#Downloading GloVe and assigning vectors to words
glove_vectors = GloVe(name='840B',dim=300)

embeddings = torch.zeros(len(src_word_to_idx)+1,300)
for word , id in src_word_to_idx.items():
    embeddings[id] = glove_vectors[word]

In [15]:
# Train-Valid split of 80-20
def split(n, ratio):
    n_val = int(ratio * n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

train_indices, val_indices = split(len(dataset_corpus), 0.2)

In [16]:
from torch.nn.utils.rnn import pad_sequence

# ----------- Batching the data -----------
def collate_fn(instn):
    src = [torch.Tensor(x[0]) for x in instn]
    tgt = torch.Tensor([x[1] for x in instn])

    padded_sent = pad_sequence(src, batch_first=True, padding_value=0)

    target = torch.cat((tgt[:, 0], tgt[:, 1],tgt[:, 2],tgt[:, 3],tgt[:, 4],tgt[:, 5],tgt[:, 6],tgt[:, 7],tgt[:, 8],tgt[:, 9],tgt[:, 10],tgt[:, 11],tgt[:, 12],tgt[:, 13],tgt[:, 14],tgt[:, 15],tgt[:, 16],tgt[:, 17],tgt[:, 18]), dim = 0)

    return (padded_sent.long(), target.long())


batch_size = 64

train_sampler   = SubsetRandomSampler(train_indices)
train_loader    = DataLoader(dataset_corpus, batch_size, sampler=train_sampler, collate_fn=collate_fn)

val_sampler     = SubsetRandomSampler(val_indices)
val_loader      = DataLoader(dataset_corpus, batch_size, sampler=val_sampler, collate_fn=collate_fn)

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda:7")

In [18]:
#Encoder

class Encoder(nn.Module):
    def __init__(self,embeddings):
        super(Encoder , self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(embeddings,padding_idx=0,freeze=False)
        self.dropout = nn.Dropout(0.2)
        self.gru = nn.GRU(input_size=300, hidden_size=128,num_layers=1, batch_first = True, bidirectional = True)

    def forward(self,src):
        x = self.embeddings(src)
        x = self.dropout(x)
        hidden , out = self.gru(x)
        return hidden

In [48]:
#Decoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.gru = nn.GRUCell(input_size=320 , hidden_size=128)
        self.lin1 = nn.Linear(128,64)
        self.lin2 = nn.Linear(64,64)
    
    def forward(self, c , st , yt):
        input = torch.cat((yt,c) , dim = 1)
        st = st[:,0,:].squeeze(dim=1)
        s = self.gru(input, st)
        x = self.lin2(F.relu(self.lin1(s)))
        return x , s


In [19]:
#Decoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.gru = nn.GRUCell(input_size=320 , hidden_size=128)
        self.lin1 = nn.Linear(128,64)
        self.lin2 = nn.Linear(64,64)
    
    def forward(self, c , st , yt):
        input = torch.cat((yt,c) , dim = 1)
        st = st[:,0,:].squeeze(dim=1)
        s = self.gru(input, st)
        x = self.lin2(F.relu(self.lin1(s)))
        return x , s


In [20]:
#Transalator

class Translator(nn.Module):
    def __init__(self,embeddings):
        super(Translator , self).__init__()
        self.encoder = Encoder(embeddings)
        self.initial_hidden = nn.Linear(256,128)
        self.attention = nn.Linear(384,1)
        self.decoder = Decoder()
    
    def forward(self, src):
        #Batch Size , Sequence Length
        B , len_seq = src.shape[0] , src.shape[1]
        # Hidden Vectors
        H = self.encoder(src)
        st = self.initial_hidden(H)

        logits = torch.zeros(B , 64).to(device)
        for i in range(19):
            Hs = torch.cat((H,st) , dim = 2)

            #Attention 
            a = torch.softmax(self.attention(Hs).squeeze() , dim=1)

            #Context
            c = torch.bmm(a.unsqueeze(dim=1) , H).squeeze(dim=1)

            yt , st = self.decoder(c , st , logits[B*i : B*(i+1)])
            logits = torch.cat((logits , yt),dim=0)
            st = st.unsqueeze(dim=1)
            st = st.repeat(1,len_seq , 1)
        
        return logits[B:]



In [21]:
model = Translator(embeddings)
model.to(device)
optimizer = torch.optim.Adam(model.parameters() , lr = 1e-3)
loss_function = F.cross_entropy

In [22]:
def exact_match(y_pred , tgt):
    B = tgt.shape[0] // 19
    correct =0
    for i in range(B):
        f =True
        for j in range(19):
            if(tgt[i+(j*B)] != y_pred[i+(j*B)]):
                f=False
                break
        if(f):
            correct +=1
    return correct , B

In [31]:
num_epochs = 20
best_val_loss = 100
best_accuracy =0
best_accuracy_em =0
for epoch in range(1,num_epochs+1):
    epoch_loss =0
    model.train()
    train_labels = []
    train_pred =[]
    correct_train , B_train =0 ,0
    #Training
    for src, tgt in tqdm(train_loader):
        optimizer.zero_grad()
        src = src.to(device)
        tgt = tgt.to(device)

        y_pred = model(src)
        loss = loss_function(y_pred , tgt)

        loss.backward()
        optimizer.step()
        epoch_loss += float(loss)

        y_pred = torch.softmax(y_pred , dim=1).argmax(dim=1)

        correct , B = exact_match(y_pred , tgt)
        correct_train +=correct
        B_train +=B

        train_labels.extend(tgt.cpu().detach().numpy())
        train_pred.extend(y_pred.cpu().detach().numpy())

    print(f'Epoch : {epoch} Training Loss : {epoch_loss/len(train_loader)} Train Accuracy : {accuracy_score(train_labels ,train_pred)*100} Train Exact-Match { (correct_train/B_train)*100}')
    # Validating
    valid_loss=0
    valid_labels = []
    valid_pred =[]
    correct_valid , B_valid = 0 , 0
    for src , tgt in tqdm(val_loader):
        src = src.to(device)
        tgt = tgt.to(device)
        


        y_pred = model(src)
        loss = loss_function(y_pred,tgt)
        valid_loss += loss

        y_pred = torch.softmax(y_pred , dim=1).argmax(dim=1)

        correct , B = exact_match(y_pred , tgt)
        correct_valid +=correct
        B_valid +=B

        valid_labels.extend(tgt.cpu().detach().numpy())
        valid_pred.extend(y_pred.cpu().detach().numpy())
    valid_loss = valid_loss/len(val_loader)
    accuracy = round(accuracy_score(valid_labels ,valid_pred)*100,4)
    accuracy_em = round((correct_valid/B_valid)*100 , 4)
    print(f'Validation Loss : {valid_loss} Validation Accuracy : {accuracy} Validation Exact-Match {accuracy_em}')
    if(accuracy_em > best_accuracy_em):
        i=epoch
        best_val_loss = valid_loss
        best_accuracy = accuracy
        best_accuracy_em = accuracy_em
        torch.save(model.state_dict() , "Transliteration Best Model.pt")

print("\nTraining Completed")

print(f"\n Best Epoch : {i} Valid Loss : {best_val_loss} Accuracy : {best_accuracy} Exact-Match : {best_accuracy_em}")




        

100%|██████████| 640/640 [00:23<00:00, 26.84it/s]


Epoch : 1 Training Loss : 0.2891474076313898 Train Accuracy : 90.35721628289474 Train Exact-Match 27.150878906249996


100%|██████████| 160/160 [00:03<00:00, 43.61it/s]


Validation Loss : 0.35275158286094666 Validation Accuracy : 88.5485 Validation Exact-Match 22.1875


100%|██████████| 640/640 [00:24<00:00, 26.26it/s]


...

Validation Loss : 0.35271385312080383 Validation Accuracy : 88.5002 Validation Exact-Match 21.875


100%|██████████| 640/640 [00:23<00:00, 27.35it/s]


Epoch : 20 Training Loss : 0.2890837023733184 Train Accuracy : 90.35952919407895 Train Exact-Match 27.248535156249996


100%|██████████| 160/160 [00:03<00:00, 49.49it/s]

Validation Loss : 0.352411687374115 Validation Accuracy : 88.5598 Validation Exact-Match 22.2559

Training Completed

 Best Epoch : 14 Valid Loss : 0.35204485058784485 Accuracy : 88.5866 Exact-Match : 22.5391


In [32]:
def SrcString(src_string):
    out =""
    for i in src_string:
        try:
            out = out + idx_to_src_word[i]
        except:
            out = out
    return out[:-1]
        
def TrgString(target):
    out=""
    for i in target:
        if i == 0:
            continue
        else :
            out = out + idx_to_tgt_word[i]
    return out
def Ouput(tgt , y_pred ,src_string, idx , B):
    src_string = src_string.cpu().detach().numpy()
    src_string = SrcString(src_string)
    target = [tgt[idx+(j*B)].item() for j in range(19)]
    target = TrgString(target)
    pred = [y_pred[idx+(j*B)].item() for j in range(19)]
    pred = TrgString(pred)
    return src_string,target , pred


In [39]:
def exactmatch_ouputmatch(y_pred,tgt,src, file):
    B = tgt.shape[0] // 19
    correct =0
    for i in range(B):
        f =True
        for j in range(19):
            if(tgt[i+(j*B)] != y_pred[i+(j*B)]):
                src_word, target , pred = Ouput(tgt,y_pred,src[i],i,B)
                file.write("\n"+ src_word + " : " +target +" : "+pred)
                f=False
                break
        if(f):
            correct +=1
    output_correct = [0 for i in range(19)]
    for i in range(B):
        for j in range(19):
            if tgt[i+(j*B)] == y_pred[i+(j*B)]:
                output_correct[j] += 1
    return correct, B, output_correct

In [40]:
def ShowOutput(y_pred,tgt,src, file):
    B = tgt.shape[0] // 19
    correct =0
    for i in range(B):
        src_word, target , pred = Ouput(tgt,y_pred,src[i],i,B)
        print(f'Target : {target}  Predicted : {pred}')
        file.write("\n"+ " : " +target +" : "+pred )

In [41]:
model = Translator(embeddings)
model.to(device)
model.load_state_dict(torch.load("/data5/home/bhargavav2/DLNLP/Assignment2/PartA/Transliteration Best Model.pt"))
model.eval()

Translator(
  (encoder): Encoder(
    (embeddings): Embedding(28, 300, padding_idx=0)
    (dropout): Dropout(p=0.2, inplace=False)
    (gru): GRU(300, 128, batch_first=True, bidirectional=True)
  )
  (initial_hidden): Linear(in_features=256, out_features=128, bias=True)
  (attention): Linear(in_features=384, out_features=1, bias=True)
  (decoder): Decoder(
    (gru): GRUCell(320, 128)
    (lin1): Linear(in_features=128, out_features=64, bias=True)
    (lin2): Linear(in_features=64, out_features=64, bias=True)
  )
)

In [43]:
val_labels = []
val_pred = []
file = open("Wrong outputs Transliteration final.txt","w")
file.write("\n Source   :   Target     :   Predicted  ")
val_epoch_loss = 0
correct_val, B_val = 0, 0
out_correct = [0 for i in range(19)]
with torch.no_grad():
    for xb, yb in tqdm(val_loader):
        xb = xb.to(device)
        yb = yb.to(device)
        y_hat = model(xb)
        loss = loss_function(y_hat, yb)

        val_epoch_loss += float(loss)

        y_hat = torch.softmax(y_hat, dim = 1).argmax(dim=1)

        correct, B, output_correct = exactmatch_ouputmatch(y_hat, yb,xb,file)
        correct_val += correct
        B_val += B
        for i in range(19):
            out_correct[i] += output_correct[i]

        val_labels.extend(yb.cpu().detach().numpy())
        val_pred.extend(y_hat.cpu().detach().numpy())

val_epoch_loss = val_epoch_loss / len(val_loader)
print("Test loss: ", val_epoch_loss)
print("Test accuracy: ", accuracy_score(val_labels, val_pred)*100)
print("Test EM: ", (correct_val/ B_val)*100)
for i in range(19):
    out_correct[i] = (out_correct[i] / B_val) * 100
print("Test accuracy position-wise: ", out_correct)
# [ 38,  16, 283,   0]
file.close()

100%|██████████| 160/160 [00:13<00:00, 11.85it/s]

Test loss:  0.34808043465018274
Test accuracy:  88.69911595394737
Test EM:  22.998046875
Test accuracy position-wise:  [94.482421875, 77.48046875, 73.916015625, 65.185546875, 65.9375, 69.716796875, 77.51953125, 84.267578125, 89.912109375, 93.837890625, 96.572265625, 98.115234375, 99.169921875, 99.501953125, 99.833984375, 99.912109375, 99.951171875, 99.970703125, 100.0]


In [44]:
val_labels = []
val_pred = []
file = open("Transilteration final.txt","w")
file.write("\n Source   :   Target     :   Predicted  ")
val_epoch_loss = 0
correct_val, B_val = 0, 0
out_correct = [0 for i in range(10)]
i =0
with torch.no_grad():
    for xb, yb in (val_loader):
        xb = xb.to(device)
        yb = yb.to(device)
        y_hat = model(xb)
        loss = loss_function(y_hat, yb)

        val_epoch_loss += float(loss)

        y_hat = torch.softmax(y_hat, dim = 1).argmax(dim=1)

        ShowOutput(y_hat, yb,xb,file)
        if(i==10):
            break
        i=i+1


Target : आगा  Predicted : आगा
Target : गंधर्वसेन  Predicted : गंधारववेेे
Target : भिक्षाटन  Predicted : भिक्षात
Target : वोटों  Predicted : वोटों
Target : वास्ते  Predicted : वासतेे
Target : महापुरुषों  Predicted : महापुरुो
Target : संदली  Predicted : संदाीी
Target : रिहाई  Predicted : रहहई
Target : प्याले  Predicted : प्याल
Target : वक्ता  Predicted : वक्त
Target : नारियां  Predicted : नारियां
Target : चक  Predicted : चक
Target : दत्तोपंत  Predicted : डा्टोांं
Target : अल्टीमेट  Predicted : अल्टीमते
Target : आश्चर्यचकित  Predicted : आश्च्््चककत
Target : सहकर्मी  Predicted : सहकरममी
Target : इलाका  Predicted : इलाका
Target : चलतीं  Predicted : चलली
Target : ढकता  Predicted : ढक्ा
Target : फारूक  Predicted : फारकक
Target : राष्ट्रगान  Predicted : राष्ट्गगण
Target : तांबे  Predicted : टैंबे
Target : बोधिवृक्ष  Predicted : बोधिव््ष
Target : मध्ययुगीन  Predicted : मध्ययुगनन
Target : वैधता  Predicted : वैधता
Target : बालकडू  Predicted : बलाकाद
Target : श्रद्धालु  Predicted : श्राााुु
Target